In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
os.walk("/kaggle/input")


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(color_codes=True)
df=pd.read_csv("../input/mengary-revenue-prediction/train.csv")
df = df.dropna(axis=0, subset=["delivery type"])
df=df.drop(["id"],axis="columns")
df=df.drop(["address code"],axis="columns")

In [ ]:
df

In [ ]:
df.info()

In [ ]:
(df["delivery date"]==df["placement date"]).unique()

In [ ]:
df["no of items"]=df["no of items"].astype("float")
# df["address code"]=df["address code"].astype("float")
df["RID"]=df["RID"].astype("float")
df.info()

In [ ]:
new_locations=pd.get_dummies(df["location"])
new_locations=new_locations.drop(["West"],axis="columns")
df=pd.concat([df,new_locations],axis="columns")
df=df.drop(["location"],axis="columns")

In [ ]:
df.head()

In [ ]:
new_segment=pd.get_dummies(df["segment"])
new_segment=new_segment.drop(["Home Office"],axis="columns")
df=pd.concat([df,new_segment],axis="columns")
df=df.drop(["segment"],axis="columns")

In [ ]:
df.info()

In [ ]:
new_class=pd.get_dummies(df["class"])
new_class=new_class.drop(["qexty"],axis="columns")
df=pd.concat([df,new_class],axis="columns")
df=df.drop(["class"],axis="columns")

In [ ]:
new_delivery_type=pd.get_dummies(df["delivery type"])
new_delivery_type=new_delivery_type.drop(["Same Day"],axis="columns")
df=pd.concat([df,new_delivery_type],axis="columns")
df=df.drop(["delivery type"],axis="columns")

In [ ]:
new_subclass=pd.get_dummies(df["sub-class"])
new_subclass=new_subclass.drop(["shorts"],axis="columns")
df=pd.concat([df,new_subclass],axis="columns")
df=df.drop(["sub-class"],axis="columns")

In [ ]:
df.head()

In [ ]:
df["departure state"].value_counts().sort_values(ascending=False).head(50)

In [ ]:
df["departure city"].value_counts().sort_values(ascending=False).head(50)

In [ ]:
top_20_1=[x for x in df["departure city"].value_counts().sort_values(ascending=False).head(30).index]
top_20_2=[x for x in df["departure state"].value_counts().sort_values(ascending=False).head(20).index]


In [ ]:
len(top_20_1)

In [ ]:
len(top_20_2)

In [ ]:
new_departure_city=pd.get_dummies(df["departure city"])
listing_city=[]
for x in new_departure_city:
    for y in top_20_1:
        if(x==y):
            listing_city.append(x)
listing_city

In [ ]:
new_departure_state=pd.get_dummies(df["departure state"])
listing_state=[]
for x in new_departure_state:
    for y in top_20_2:
        if(x==y):
            listing_state.append(x)
listing_state

In [ ]:
new_departure_city=new_departure_city[listing_city]
new_departure_city

In [ ]:
new_departure_state=new_departure_state[listing_state]
new_departure_state

In [ ]:
df=df.drop(["departure state"],axis="columns")
df=df.drop(["departure city"],axis="columns")

In [ ]:
df=pd.concat([df,new_departure_state],axis="columns")
df=pd.concat([df,new_departure_city],axis="columns")

In [ ]:
df["delivery date"] = pd.to_datetime(df["delivery date"])
df["placement date"] = pd.to_datetime(df["placement date"])
import datetime as dt
delivery_date=df["delivery date"].map(dt.datetime.toordinal)
placement_date=df["placement date"].map(dt.datetime.toordinal)
df=df.drop(["delivery date"],axis="columns")
df=df.drop(["placement date"],axis="columns")
df=pd.concat([df,delivery_date],axis="columns")
df=pd.concat([df,placement_date],axis="columns")

In [ ]:
df[["placement date"]]

In [ ]:
df=df.astype("float")

In [ ]:
df.info()

In [ ]:
inputs=df.drop(["profit"],axis="columns")

In [ ]:
target=df["profit"]

In [ ]:
inputs.info()

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(inputs,target,test_size=0.2)
# from sklearn.ensemble import RandomForestRegressor
# model = RandomForestRegressor(n_estimators=500,random_state=1)
# model.fit(inputs,target)
# model.score(x_test, y_test)

In [ ]:
# from xgboost import XGBRegressor
# my_model = XGBRegressor(n_estimators=500,learning_rate:0.01,gamma:1,)
# my_model.fit(x_train, y_train)
# my_model.score(x_test,y_test)


In [ ]:
# from sklearn.linear_model import Lasso
# my_model=Lasso(normalize=True)
# my_model.fit(x_train,y_train)
# my_model.score(x_test,y_test)

In [ ]:
# from xgboost import XGBRegressor
# a=cross_val_score(XGBRegressor(n_estimators=4000),x_train,y_train,cv=5)
# a.mean()

In [ ]:
# from sklearn.ensemble import RandomForestRegressor
# a=cross_val_score(RandomForestRegressor(n_estimators=1000),x_train,y_train,cv=5)
# a.mean()

In [ ]:
from xgboost import XGBRegressor
model=XGBRegressor(n_estimators=500)
model.fit(x_train,y_train)
model.score(x_test,y_test)

In [ ]:
test_me=pd.read_csv("../input/helloka/Best.csv")

In [ ]:
csv=model.predict(test_me)

In [ ]:
test_id=pd.read_csv("../input/mengary-revenue-prediction/test.csv")
test_id=test_id["id"]

In [ ]:
test_id

In [ ]:
csv = pd.DataFrame(csv, columns =["profit"]) 


In [ ]:
csv=pd.concat([test_id,csv],axis="columns")

In [ ]:
csv.to_csv('GoodNight2.csv') 

In [ ]:
# inputs